# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
import os

%reload_ext dotenv
os.chdir('../../01_materials/labs')
%dotenv .env

In [2]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

c:\Users\Administrator\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [26]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
PRICE_DATA

parquet_files = glob(os.path.join(PRICE_DATA, '*/*/*.parquet'))
parquet_files
#dd_px = dd.read_parquet(parquet_files).set_index("ticker")
dd_px = dd.read_parquet(parquet_files[:1]).set_index("ticker")


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [27]:
# Write your code below.

import numpy as np

dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(    
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(
    Adj_Close = lambda x: x['Close']  
).assign(
    Adj_Close_lag_1 = lambda x: x['Adj_Close'].shift(1)
).assign(
    returns = lambda x: x['Close'] / x['Close_lag_1'] - 1
).assign(
    hi_lo_range = lambda x: x['High'] - x['Low']
))

C:\Users\Administrator\AppData\Local\Temp\ipykernel_19048\2613206845.py:5: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [28]:
# Write your code below.

#dd_feat = dd_feat.compute()
dd_feat = dd_feat.repartition(npartitions=1000000).compute()
dd_feat['rolling_avg_return'] = dd_feat['returns'].rolling(10).mean()
dd_feat.head()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,Adj_Close,Adj_Close_lag_1,returns,hi_lo_range,rolling_avg_return
ticker,,,,,,,,,,,,,,,,
A,2000-01-03,56.330471,56.464592,48.193848,51.502148,43.613010,4674353,Health Care,Life Sciences Tools & Services,2000,NaN,51.502148,NaN,NaN,8.270744,NaN
A,2000-01-04,48.730328,49.266811,46.316166,47.567955,40.281467,4765083,Health Care,Life Sciences Tools & Services,2000,51.502148,47.567955,51.502148,-0.076389,2.950645,NaN
A,2000-01-05,47.389126,47.567955,43.141991,44.617310,37.782791,5758642,Health Care,Life Sciences Tools & Services,2000,47.567955,44.617310,47.567955,-0.062030,4.425964,NaN
A,2000-01-06,44.080830,44.349072,41.577251,42.918453,36.344173,2534434,Health Care,Life Sciences Tools & Services,2000,44.617310,42.918453,44.617310,-0.038076,2.771820,NaN
A,2000-01-07,42.247852,47.165592,42.203148,46.494991,39.372860,2819626,Health Care,Life Sciences Tools & Services,2000,42.918453,46.494991,42.918453,0.083333,4.962444,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

Answer: It was not necessary as Dask also supports rolling window operations, so it could perform the rolling average return calculation directly in Dask without converting to a pandas DataFrame.

+ Would it have been better to do it in Dask? Why?

Answer: It can be more efficient to perform this operation directly in Dask and also more scalable for large datasets. Dask's ability to handle larger-than-memory computations and distribute the workload makes it a better choice for these kinds of tasks.

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.